In [ ]:
import os
from datetime import date
from datetime import datetime
import pandas as pd
import cx_Oracle
import schedule
import time

def clean_and_transform_data(directory):
    files = os.listdir(directory)

    latest_date = datetime(1900, 1, 1)  # Initialize with a very old date
    for file in files:
        if file.endswith('.xlsx'):
            file_date = datetime.strptime(file.split('.')[0], '%Y-%m-%d')
            if file_date > latest_date:
                latest_date = file_date

    print(latest_date)
    filtered_files = [file for file in files if file.endswith('.xlsx') and datetime.strptime(file.split('.')[0], '%Y-%m-%d') >= latest_date]

    dfs = []
    for file in filtered_files:
        data = pd.read_excel(os.path.join(directory, file))
        data.columns = data.iloc[4]
        data = data[5:]
        data = data[data['Дата покупки'] != 'Дата покупки']
        
        data['DATE_VALUE'] = os.path.splitext(file)[0]
        
        dfs.append(data)

    data = pd.concat(dfs, ignore_index=True)


    data.columns=['PURCHASE_DATE', 'CATEGORY', 'TICKER', 'NAME_SEC', 'TYPE_SEC',
           'TRANSACTION_NUMBER', 'TRANSACTION_NAME', 'ISSUER', 'ISIN_NIN',
           'CURRENCY_NOMINAL', 'PAYMENT_CURRENCY', 'NOMINAL_PER_UNIT', 'QUANTITY',
           'NOMINAL_VALUE_IN_CURRENCY', 'NOMINAL_VALUE_KZT', 'DISCOUNT_PREM',
           'DISCOUNT_PREM_KZT', 'AMORT_DISCOUNT_PREM', 'AMORT_DISCOUNT_PREM_KZT',
           'MARK_TO_MARKET_CURR', 'MARK_TO_MARKET_KZT', 'ACCRUED_INTEREST_CURR',
           'ACCRUED_INTEREST_CURR_KZT', 'CLEAN_PRICE', 'CLEAN_PRICE_CURR',
           'DIRTY_PRICE_KZT', 'DIRTY_PRICE_CURR', 'PURCHASE_PRICE',
           'AMORTIZED_PRICE', 'MARKET_PRICE', 'AGENCY', 'TYPE', 'CURRENCY', 'YTM',
           'COUPON_RATE', 'COUPON_TYPE', 'BASIS_CODE', 'BASIS_NAME', 'ISSUE_DATE',
           'MATURITY', 'LAST_COUPON_PAYMENT_DATE', 'DATE_OF_NEXT_PAYMENT',
           'DAYS_TO_COUPON_PAYMENT', 'DAYS_TO_MATURITY', 'ECONOMIC_SECTOR',
           'RESIDENCY_TYPE', 'STORAGE_LOCATION', 'DATE_VALUE']

    columns_to_convert = ['QUANTITY', 'NOMINAL_VALUE_IN_CURRENCY', 'NOMINAL_VALUE_KZT', 'DISCOUNT_PREM',
                          'DISCOUNT_PREM_KZT', 'AMORT_DISCOUNT_PREM', 'AMORT_DISCOUNT_PREM_KZT',
                          'MARK_TO_MARKET_CURR', 'MARK_TO_MARKET_KZT', 'ACCRUED_INTEREST_CURR',
                          'ACCRUED_INTEREST_CURR_KZT', 'CLEAN_PRICE', 'CLEAN_PRICE_CURR',
                          'DIRTY_PRICE_KZT', 'DIRTY_PRICE_CURR', 'PURCHASE_PRICE', 'AMORTIZED_PRICE',
                          'MARKET_PRICE', 'YTM', 'COUPON_RATE', 'DAYS_TO_COUPON_PAYMENT', 'DAYS_TO_MATURITY']

    data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

    def try_convert_to_datetime(series):
        try:
            return pd.to_datetime(series)
        except ValueError:
            return series

    for col in data[['PURCHASE_DATE','ISSUE_DATE','MATURITY','LAST_COUPON_PAYMENT_DATE','DATE_OF_NEXT_PAYMENT','DATE_VALUE']]:
        data[col] = try_convert_to_datetime(data[col])

    return data

def insert_data_to_oracle(data):
    dsn = cx_Oracle.makedsn('10.23.13.62', '1521', 'OPERRISKDB')
    connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)
    cursor = connection.cursor()

    table_name = 'NELLI_CB_NEW_VERSION'

    try:
        for _, row in data.iterrows():
            placeholders = ', '.join([':' + str(i+1) for i in range(len(row))])
            values = []
            for val in row:
                if pd.isnull(val):
                    values.append(None)
                elif isinstance(val, pd.Timestamp):  # Check if the value is a pandas Timestamp (datetime)
                    values.append(val.to_pydatetime())
                else:
                    values.append(str(val))  # Convert to string for non-datetime columns
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(query, values)

        connection.commit()
        print("Data inserted successfully.")
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Error {error.code}: {error.message}")
        print(f"Problematic query: {query}")

    cursor.close()
    connection.close()

today = date.today()
directory = fr"W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Алтынай\ЦБ\2024\{str(today.month).zfill(2)}"

def process_data_and_insert():
    new_data = clean_and_transform_data(directory)
    insert_data_to_oracle(new_data)

# schedule.every().day.at("17:51").do(process_data_and_insert)

# while True:
#     schedule.run_pending()
#     time.sleep(60)  # Check every 60 seconds


